In [1]:
import re
import nltk
from sklearn import svm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from google_trans_new import google_translator  
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
fg_df = pd.read_csv('D:/Users/ehern/Desktop/Machine Learning/Project/fall_guys_metacritic_reviews.csv')

fg_df.head()

,username,review_type,published_date,score,votes,review_text,profile_url,platform
0,Fen1ma,user,"Aug 26, 2020",6.0,0.0,"на данный момент не советую покупать игру, ещё...",https://www.metacritic.com/user/Fen1ma,pc
1,Locke_VI,user,"Aug 25, 2020",3.0,0.0,Fall Guys was a game I was really looking forw...,https://www.metacritic.com/user/Locke_VI,pc
2,Crisener,user,"Aug 25, 2020",8.0,0.0,You gotta love it. The aesthetics are very col...,https://www.metacritic.com/user/Crisener,pc
3,EnigmaXtreme,user,"Aug 25, 2020",7.0,0.0,When we covered Fall Guys last week during it’...,https://www.metacritic.com/user/EnigmaXtreme,pc
4,nitorita,user,"Aug 23, 2020",7.0,0.0,I like how Fall Guys adapted Mario Party and m...,https://www.metacritic.com/user/nitorita,pc


In [3]:
fg_df.drop(columns=['username','review_type','published_date','published_date', 'votes', 'profile_url', 'platform'],inplace=True)

In [4]:
noNull = fg_df.dropna()

In [5]:
# for i in noNull['review_text']:
#     detector = google_translator()  
#     detect_result = detector.detect(i)
#     print(detect_result)

In [6]:
translations = {}
for i in noNull['review_text']:
    translator = google_translator()
    translations[i] = translator.translate(i, lang_tgt='en')
    
noNull.replace(translations, inplace = True)

# check translation
noNull

C:\Users\ehern\anaconda3\lib\site-packages\pandas\core\frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,score,review_text
0,6.0,At the moment I do not advise you to buy the g...
1,3.0,Fall Guys was a game I was really looking forw...
2,8.0,You gotta love it. The aesthetics are very col...
3,7.0,When we covered Fall Guys last week during it’...
4,7.0,I like how Fall Guys adapted Mario Party and m...
...,...,...
349,1.0,Fall guys is a great game that is destroyed by...
350,9.0,"Simple and fun, very easy to understand gamepl..."
351,8.0,"HIVALLY FOR ALL ages, and when you only have a..."
352,9.0,This game is fun but when you play with friend...


In [7]:
translated_df = noNull.copy()

In [8]:
translated_df.isnull().sum()

score          0
review_text    0
dtype: int64

In [9]:
translated_df.describe()

,score
count,329.000000
mean,6.802432
std,3.340512
min,0.000000
25%,5.000000
50%,8.000000
75%,10.000000
max,10.000000


In [10]:
translated_df['score'].value_counts()

10.0    83
8.0     61
9.0     50
0.0     34
7.0     26
6.0     20
5.0     15
2.0     12
1.0     11
3.0     10
4.0      7
Name: score, dtype: int64

In [11]:
translated_df['review_text'] = translated_df['review_text'].str.replace('[^\w\s]','')

In [12]:
translated_df['review_text'] = translated_df['review_text'].str.lower()

In [13]:
translated_df.head()

,score,review_text
0,6.0,at the moment i do not advise you to buy the g...
1,3.0,fall guys was a game i was really looking forw...
2,8.0,you gotta love it the aesthetics are very colo...
3,7.0,when we covered fall guys last week during its...
4,7.0,i like how fall guys adapted mario party and m...


In [14]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
def tokenize(text):
    tokens = re.split('\W+', str(text))
    return tokens

translated_df['review_text'] = translated_df['review_text'].apply(lambda w: tokenize(w))
translated_df.head()

,score,review_text
0,6.0,"[at, the, moment, i, do, not, advise, you, to,..."
1,3.0,"[fall, guys, was, a, game, i, was, really, loo..."
2,8.0,"[you, gotta, love, it, the, aesthetics, are, v..."
3,7.0,"[when, we, covered, fall, guys, last, week, du..."
4,7.0,"[i, like, how, fall, guys, adapted, mario, par..."


In [16]:
stopwords = nltk.corpus.stopwords.words('english')
def removeStopWords(text):
    cleanText = [w for w in text if w not in stopwords]
    return cleanText

translated_df['review_text'] = translated_df['review_text'].apply(lambda w: removeStopWords(w))
translated_df.head()

,score,review_text
0,6.0,"[moment, advise, buy, game, still, raw, br, mi..."
1,3.0,"[fall, guys, game, really, looking, forward, f..."
2,8.0,"[gotta, love, aesthetics, colorful, charming, ..."
3,7.0,"[covered, fall, guys, last, week, beta, talked..."
4,7.0,"[like, fall, guys, adapted, mario, party, made..."


In [17]:
# def listToString(text): 
#     str1 = " " 
#     return (str1.join(text))
        
# translated_df['review_text'] = translated_df['review_text'].apply(lambda w: listToString(w))
# translated_df.head()

In [18]:
# vectorizer = TfidfVectorizer()

In [19]:
import gensim

In [20]:
# X = vectorizer.fit_transform(translated_df['review_text'])
# y = translated_df['score']

In [21]:
# from nltk.corpus import brown
# model = gensim.models.Word2Vec(brown.sents())

In [22]:
from gensim.models import Word2Vec

model = Word2Vec(translated_df['review_text'], vector_size = 329, min_count=1, workers=4)

In [23]:
model.save('brown.embedding')
new_model = gensim.models.Word2Vec.load('brown.embedding')

In [24]:
# print(similar_words)

In [47]:
X = np.array(new_model.wv['bad'], dtype=float).reshape(-1, 1)
y = translated_df['score']
# y = translated_df['score']

In [48]:
X.shape

(329,)

In [49]:
y.shape

(329,)

In [50]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_predict)
    return accuracy

In [51]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.28, random_state=31)

print("Accuracy:", get_score(DecisionTreeClassifier(), X_train, X_test, y_train, y_test))

cv_score_dt = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)

print("\nCross Validation Accuracy:", (cv_score_dt.mean()))

ValueError: Expected 2D array, got 1D array instead:
array=[-5.63855888e-03 -1.71760411e-03  5.08151716e-03 -5.84507314e-03
  2.48045870e-03 -1.19174300e-02 -6.44919463e-03 -4.86574788e-03
  3.84467491e-03 -6.48372667e-03  8.37015250e-05  4.78749303e-03
 -5.47698606e-03 -6.24768995e-03 -4.92127566e-03  7.03952461e-03
 -7.20075704e-03 -2.94243870e-03  2.49006553e-03 -2.97569670e-03
  1.22954766e-03  5.81780029e-03  6.44995482e-04  2.39688228e-03
 -1.30235590e-03 -2.27511744e-03 -8.85553382e-05 -7.71147851e-03
  9.66342224e-04 -9.75726289e-04  1.35125527e-02  2.17120233e-03
 -3.39118857e-03  1.21005194e-03  1.22184921e-02  8.32272787e-03
 -7.01932237e-04  8.18643533e-03 -7.28494488e-03  2.32055341e-03
 -6.53453823e-03  8.35912488e-03  1.06068710e-02 -4.84854029e-03
 -8.86216294e-03  5.80624957e-03 -1.26822793e-03 -2.99557485e-03
 -4.31758212e-03  2.83447164e-03 -5.34134917e-03 -2.82279076e-03
  2.19949634e-05 -1.08776158e-02  1.05558774e-02 -1.79661345e-02
 -6.30994281e-03  8.85926187e-0

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)

print("Accuracy:", get_score(LogisticRegression(), X_train, X_test, y_train, y_test))

cv_score_lr = cross_val_score(LogisticRegression(), X, y, cv=5)

print("\nCross Validation Accuracy:", (cv_score_lr.mean()))

ValueError: Expected 2D array, got 1D array instead:
array=[-6.44919463e-03 -4.86574788e-03  3.84467491e-03 -6.48372667e-03
  8.37015250e-05  4.78749303e-03 -5.47698606e-03 -6.24768995e-03
 -4.92127566e-03  7.03952461e-03 -7.20075704e-03 -2.94243870e-03
  2.49006553e-03 -2.97569670e-03  1.22954766e-03  5.81780029e-03
  6.44995482e-04  2.39688228e-03 -1.30235590e-03 -2.27511744e-03
 -8.85553382e-05 -7.71147851e-03  9.66342224e-04 -9.75726289e-04
  1.35125527e-02  2.17120233e-03 -3.39118857e-03  1.21005194e-03
  1.22184921e-02  8.32272787e-03 -7.01932237e-04  8.18643533e-03
 -7.28494488e-03  2.32055341e-03 -6.53453823e-03  8.35912488e-03
  1.06068710e-02 -4.84854029e-03 -8.86216294e-03  5.80624957e-03
 -1.26822793e-03 -2.99557485e-03 -4.31758212e-03  2.83447164e-03
 -5.34134917e-03 -2.82279076e-03  2.19949634e-05 -1.08776158e-02
  1.05558774e-02 -1.79661345e-02 -6.30994281e-03  8.85926187e-03
 -5.63711906e-03 -9.72403493e-03 -7.14420946e-03 -8.47204868e-03
 -2.51607783e-03  4.96489601e-0

In [41]:
# from sklearn.metrics import classification_report

# print(classification_report(y_test, y_predict_dt))

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, random_state=31)

print("Accuracy:", get_score(RandomForestClassifier(), X_train, X_test, y_train, y_test))

cv_score_rf = cross_val_score(RandomForestClassifier(), X, y, cv=5)

print("\nCross Validation Accuracy:", (cv_score_rf.mean()))

Accuracy: 0.16071428571428573

Cross Validation Accuracy: 0.17025641025641028


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.26, random_state=31)

print("Accuracy:", get_score(svm.SVC(), X_train, X_test, y_train, y_test))

cv_score_svm = cross_val_score(svm.SVC(), X, y, cv=5)

print("\nCross Validation Accuracy:", (cv_score_svm.mean()))

Accuracy: 0.27906976744186046

Cross Validation Accuracy: 0.2552913752913753


In [44]:
# from sklearn.model_selection import StratifiedKFold
# kf = StratifiedKFold(n_splits = 5)